## Update Defective Case Dataset
Current SOP for recording defects at final inspection dictates that only cases found to have defects are recorded in Final Inspection Dataset. A limitation was discovered: as instances where items run without defects are not recorded, gross percentage defective calculation (average defect performace of all instances of an item) is often far higher than it should be. Using a historical dataset of all production instances of an item, identify which JobItem events have not been recorded (due to lack of defects discovered) and update current dataset to include these 0% defective instances.

---
#### Importing packages

In [2]:
import pandas as pd

*Each production order ("Job") contains one or more products ("Items"). Therefore it is possible to aggregate over item to understand the gross behavior and performance of a product. However, since we are interested in every instance of an item within a job, we have to create a unique identifier for that event. I chose to simply use the concatenation of "Job" and "Item".*

#### Original Dataset

In [69]:
# read in original dataset
Original_DF = pd.read_csv("original.csv", header = 0, encoding = 'unicode_escape', engine ='python')

# force numeric ID columns to be of type str
datatypesdict = {'Job #' : str, 'Item' : str}
Original_DF = Original_DF.astype(datatypesdict)

# Create New Column for JobItem key
Original_DF["JobItem"] = Original_DF["Job #"] + Original_DF["Item"]

# check out top 5 rows
Original_DF.head()

,Date,Customer,Job #,Item,AQL Shift,Inspection Station (Gluer),Total Pieces,Defect Code,Defect Description,Defective Samples,Total Samples,Total Estimated Defective,Department of Origin,Machine of Origin,JobItem
0,9/29/2021,Titleist,411647,8607.0,1,2,300,P6,Coating,5,20,75.0,PRINTING,KBA RAPIDA130 P3,4116478607.0
1,9/29/2021,Titleist,411647,8607.0,1,2,300,P6,Coating,2,20,30.0,PRINTING,KBA RAPIDA130 P3,4116478607.0
2,9/29/2021,Titleist,411647,8607.0,1,2,300,P6,Coating,2,20,30.0,PRINTING,KBA RAPIDA130 P3,4116478607.0
3,9/29/2021,Titleist,411647,8607.0,1,2,300,P3,Marks,5,20,75.0,PRINTING,KBA RAPIDA130 P3,4116478607.0
4,9/29/2021,Titleist,411647,8607.0,1,2,300,NaN,Peeling,14,20,210.0,Other,BOBST 130ER DC3,4116478607.0


#### Historical Data from ProdDB

In [164]:
# read exported sql query from prod DB containting job and item pairings
Master_Job_Set = pd.read_csv('JobITemMaster.csv', header = 0, encoding = 'unicode_escape', engine ='python')
Master_Job_Set = Master_Job_Set.astype(datatypesdict)

# Master_Job_Set.dtypes

# New Column for Join
Master_Job_Set["JobItem"] = Master_Job_Set["Job #"] + Master_Job_Set["Item"]

Master_Job_Set.head()

,Item,Job #,JobItem
0,7.0,380070,3800707.0
1,8.0,380105,3801058.0
2,12.0,380105,38010512.0
3,20.0,371024,37102420.0
4,21.0,371024,37102421.0


#### Merging the two

In [168]:

# Left Join Dataframes
Join = pd.merge(Master_Job_Set, Original_DF, on="JobItem", how="left")

# Fill columns for target data set with source values
Join['Total Estimated Defective'].fillna(0, inplace=True)
Join['Job #_y'].fillna(Join['Job #_x'], inplace=True)
Join['Item_y'].fillna(Join['Item_x'], inplace=True)

# Drop extra columns
Join = Join.drop(columns= ['Item_x', 'Job #_x', 'JobItem'])

# Convert Left Join -> Full Outer Join IS NULL aka all in T1 where no match with T2
Join = Join[Join['AQL Shift'].isna()]

#Limit scope of dataset to only Jobs that ran after Defect Monitoring started, prevent old production events of the item from pulling down the metric
Joindtypes = {'Job #_y' : int}
Join = Join.astype(Joindtypes)
Join = Join[Join['Job #_y'] > 400891]

# View changes for verification
Join

# Export new Dataset to .csv
Join.to_csv("./UpdatedDataset.csv", index=0)

,Date,Customer,Job #_y,Item_y,AQL Shift,Inspection Station (Gluer),Total Pieces,Defect Code,Defect Description,Defective Samples,Total Samples,Total Estimated Defective,Department of Origin,Machine of Origin
297,NaN,NaN,401039,224.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
298,NaN,NaN,401122,224.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
299,NaN,NaN,401456,224.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
300,NaN,NaN,401614,224.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
301,NaN,NaN,401827,224.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,NaN,NaN,421446,9956.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
26453,NaN,NaN,421446,9957.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
26454,NaN,NaN,421446,9958.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
26455,NaN,NaN,421446,9959.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
